In [5]:
import multiprocessing

def worker(num):
    """thread worker function"""
    print ('Worker:', num)
    return

if __name__ == '__main__':
    jobs = []
    for i in range(5):
        p = multiprocessing.Process(target=worker, args=(i,))
        jobs.append(p)
        p.start()

Worker: 3
Worker: 1
Worker: 0
Worker: 2
Worker: 4


In [8]:
import multiprocessing
import time

def worker():
    name = multiprocessing.current_process().name
    print (name, 'Starting')
    time.sleep(2)
    print (name, 'Exiting')

def my_service():
    name = multiprocessing.current_process().name
    print (name, 'Starting')
    time.sleep(3)
    print (name, 'Exiting')

if __name__ == '__main__':
    service = multiprocessing.Process(name='my_service', target=my_service)
    worker_1 = multiprocessing.Process(name='worker 1', target=worker)
    worker_2 = multiprocessing.Process(target=worker) # use default name

    worker_1.start()
    worker_2.start()
    service.start()

worker 1 Starting
Process-26 Starting
my_service Starting
worker 1Process-26my_service Exiting
 Exiting
 Exiting


In [9]:
import sys

def daemon():
    p = multiprocessing.current_process()
    print ('Starting:', p.name, p.pid)
    sys.stdout.flush()
    time.sleep(2)
    print ('Exiting :', p.name, p.pid)
    sys.stdout.flush()

def non_daemon():
    p = multiprocessing.current_process()
    print ('Starting:', p.name, p.pid)
    sys.stdout.flush()
    print ('Exiting :', p.name, p.pid)
    sys.stdout.flush()

if __name__ == '__main__':
    d = multiprocessing.Process(name='daemon', target=daemon)
    d.daemon = True

    n = multiprocessing.Process(name='non-daemon', target=non_daemon)
    n.daemon = False

    d.start()
    time.sleep(1)
    n.start()


Process-29 Starting
worker 1 Starting
my_service Starting
Process-29worker 1my_service Exiting
 Exiting
 Exiting
Starting: daemon 67461


In [10]:
def daemon():
    print ('Starting:', multiprocessing.current_process().name)
    time.sleep(2)
    print ('Exiting :', multiprocessing.current_process().name)

def non_daemon():
    print ('Starting:', multiprocessing.current_process().name)
    print ('Exiting :', multiprocessing.current_process().name)

if __name__ == '__main__':
    d = multiprocessing.Process(name='daemon', target=daemon)
    d.daemon = True

    n = multiprocessing.Process(name='non-daemon', target=non_daemon)
    n.daemon = False

    d.start()
    time.sleep(1)
    n.start()

    d.join()
    n.join()

Starting: non-daemon 67468
Exiting : daemon 67461
Exiting : non-daemon 67468
Starting: daemon
Starting: non-daemon
Exiting :Exiting : non-daemon
 daemon


In [13]:
def exit_error():
    sys.exit(1)

def exit_ok():
    return

def return_value():
    return 1

def raises():
    raise RuntimeError('There was an error!')

def terminated():
    time.sleep(3)

if __name__ == '__main__':
    jobs = []
    for f in [exit_error, exit_ok, return_value, raises, terminated]:
        print ('Starting process for', f)
        j = multiprocessing.Process(target=f, name=f)
        jobs.append(j)
        j.start()
        
    jobs[-1].terminate()

    for j in jobs:
        j.join()
        print ('%s.exitcode = %s' % (j.name, j.exitcode))

Starting process for <function exit_error at 0x103d5ed08>
Starting process for <function exit_ok at 0x103d5ed90>
Starting process for <function return_value at 0x103ca9378>
Starting process for <function raises at 0x103ca9400>
Starting process for <function terminated at 0x103ca9488>
<function exit_error at 0x103d5ed08>.exitcode = 1
<function exit_ok at 0x103d5ed90>.exitcode = 0
<function return_value at 0x103ca9378>.exitcode = 0
<function raises at 0x103ca9400>.exitcode = 1
<function terminated at 0x103ca9488>.exitcode = -15


Process <function raises at 0x103ca9400>:
Traceback (most recent call last):
  File "/Users/sameera/mc/envs/py3k/lib/python3.4/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/Users/sameera/mc/envs/py3k/lib/python3.4/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-13-2060248865dd>", line 11, in raises
    raise RuntimeError('There was an error!')
RuntimeError: There was an error!


In [12]:
def slow_worker():
    print ('Starting worker')
    time.sleep(0.1)
    print ('Finished worker')

if __name__ == '__main__':
    p = multiprocessing.Process(target=slow_worker)
    print ('BEFORE:', p, p.is_alive())
    
    p.start()
    print ('DURING:', p, p.is_alive())
    
    p.terminate()
    print ('TERMINATED:', p, p.is_alive())

    p.join()
    print ('JOINED:', p, p.is_alive())

BEFORE: <Process(Process-34, initial)> False
DURING: <Process(Process-34, started)> True
TERMINATED: <Process(Process-34, started)> True
JOINED: <Process(Process-34, stopped[SIGTERM])> False


In [15]:
import logging
import sys

def worker():
    print ('Doing some work')
    sys.stdout.flush()

if __name__ == '__main__':
    multiprocessing.log_to_stderr(logging.DEBUG)
    p = multiprocessing.Process(target=worker)
    p.start()
    p.join()
    

Doing some work


[INFO/Process-40] child process calling self.run()
[INFO/Process-40] process shutting down
[DEBUG/Process-40] running all "atexit" finalizers with priority >= 0
[DEBUG/Process-40] running the remaining "atexit" finalizers
[INFO/Process-40] process exiting with exitcode 0


In [19]:
class Worker(multiprocessing.Process):

    def run(self):
        print ('In %s' % self.name)
        return

if __name__ == '__main__':
    jobs = []
    for i in range(5):
        p = Worker()
        jobs.append(p)
        print ("jobs", jobs)
        p.start()
    for j in jobs:
        j.join()
        print ("j", j)

jobs [<Worker(Worker-56, initial)>]
jobs [<Worker(Worker-56, started)>, <Worker(Worker-57, initial)>]
jobs [<Worker(Worker-56, started)>, <Worker(Worker-57, started)>, <Worker(Worker-58, initial)>]
jobs [<Worker(Worker-56, started)>, <Worker(Worker-57, started)>, <Worker(Worker-58, started)>, <Worker(Worker-59, initial)>]
jobs [<Worker(Worker-56, started)>, <Worker(Worker-57, started)>, <Worker(Worker-58, started)>, <Worker(Worker-59, started)>, <Worker(Worker-60, initial)>]


[INFO/Worker-56] child process calling self.run()


j <Worker(Worker-56, stopped)>
j <Worker(Worker-57, stopped)>
j <Worker(Worker-58, stopped)>
j <Worker(Worker-59, stopped)>
j <Worker(Worker-60, stopped)>
In Worker-56
In Worker-58
In Worker-57
In Worker-59
In Worker-60


[INFO/Worker-57] child process calling self.run()
[INFO/Worker-59] child process calling self.run()
[INFO/Worker-56] process shutting down
[INFO/Worker-58] child process calling self.run()
[INFO/Worker-60] child process calling self.run()
[INFO/Worker-57] process shutting down
[INFO/Worker-59] process shutting down
[DEBUG/Worker-56] running all "atexit" finalizers with priority >= 0
[INFO/Worker-58] process shutting down
[INFO/Worker-60] process shutting down
[DEBUG/Worker-57] running all "atexit" finalizers with priority >= 0
[DEBUG/Worker-59] running all "atexit" finalizers with priority >= 0
[DEBUG/Worker-56] running the remaining "atexit" finalizers
[DEBUG/Worker-58] running all "atexit" finalizers with priority >= 0
[DEBUG/Worker-60] running all "atexit" finalizers with priority >= 0
[DEBUG/Worker-57] running the remaining "atexit" finalizers
[DEBUG/Worker-59] running the remaining "atexit" finalizers
[INFO/Worker-56] process exiting with exitcode 0
[DEBUG/Worker-58] running the r

In [23]:
from functools import partial
from multiprocessing.pool import Pool

if __name__ == "__main__":
   ts = time()
   client_id = os.getenv('IMGUR_CLIENT_ID')
   if not client_id:
       raise Exception("Couldn't find IMGUR_CLIENT_ID environment variable!")
   download_dir = setup_download_dir()
   links = [l for l in get_links(client_id) if l.endswith('.jpg')]
   download = partial(download_link, download_dir)
   with Pool(8) as p:
       p.map(download, links)
   print('Took {}s'.format(time() - ts))

TypeError: 'module' object is not callable

In [22]:
run main

ERROR: File `'main.py'` not found.


In [27]:
from multiprocessing import Process
def fn(x):
    print (x*x)
    return x*x

if __name__ == '__main__':
    p = Process(target=fn, args=([2]))
    p.start()
    p.join()
    print (p)

<Process(Process-63, stopped)>
4


[INFO/Process-63] child process calling self.run()
[INFO/Process-63] process shutting down
[DEBUG/Process-63] running all "atexit" finalizers with priority >= 0
[DEBUG/Process-63] running the remaining "atexit" finalizers
[INFO/Process-63] process exiting with exitcode 0


In [29]:
try:
    import urllib.request as urllib2
except ImportError:
    import urllib2
from multiprocessing.dummy import Pool as ThreadPool 

urls = [
  'http://www.python.org', 
  'http://www.python.org/about/',
  'http://www.onlamp.com/pub/a/python/2003/04/17/metaclasses.html',
  'http://www.python.org/doc/',
  'http://www.python.org/download/',
  'http://www.python.org/getit/',
  'http://www.python.org/community/',
  'https://wiki.python.org/moin/',
  'http://planet.python.org/',
  'https://wiki.python.org/moin/LocalUserGroups',
  'http://www.python.org/psf/',
  'http://docs.python.org/devguide/',
  'http://www.python.org/community/awards/'
  # etc.. 
  ]

# Make the Pool of workers
pool = ThreadPool(4) 
# Open the urls in their own threads
# and return the results
results = pool.map(urllib2.urlopen, urls)
#close the pool and wait for the work to finish 
pool.close() 
pool.join() 

[DEBUG/MainProcess] added worker
[DEBUG/MainProcess] added worker
[DEBUG/MainProcess] added worker
[DEBUG/MainProcess] added worker
[DEBUG/MainProcess] closing pool
[DEBUG/MainProcess] joining pool
[DEBUG/MainProcess] worker handler exiting
[DEBUG/MainProcess] task handler got sentinel
[DEBUG/MainProcess] task handler sending sentinel to result handler
[DEBUG/MainProcess] task handler sending sentinel to workers
[DEBUG/MainProcess] task handler exiting
[DEBUG/MainProcess] worker got sentinel -- exiting
[DEBUG/MainProcess] worker exiting after 4 tasks
[DEBUG/MainProcess] worker got sentinel -- exiting
[DEBUG/MainProcess] worker exiting after 3 tasks
[DEBUG/MainProcess] worker got sentinel -- exiting
[DEBUG/MainProcess] worker exiting after 3 tasks
[DEBUG/MainProcess] worker got sentinel -- exiting
[DEBUG/MainProcess] worker exiting after 3 tasks
[DEBUG/MainProcess] result handler got sentinel
[DEBUG/MainProcess] result handler exiting: len(cache)=0, thread._state=0
